Ejemplo tomado de https://github.com/StrikingLoo/Cats-and-dogs-classifier-tensorflow-CNN/blob/master/Convolutional_experiments.ipynb. El ejemplo ha sido simplificado y adaptado para el curso introduccion de aprendizaje de maquinas de la Maestria en Ingenieria de la Universidad del Magdalena

In [2]:
import tensorflow as tf
import seaborn as sns
import numpy as np

from PIL import Image
import glob
from collections import defaultdict
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
dir_path_cat = './PetImages\\Cat/*'
dir_path_dog = './PetImages\\Dog/*'
IMG_SIZE = (94, 125)

In [4]:
def pixels_from_path(file_path):
    im = Image.open(file_path)
    im = im.resize(IMG_SIZE)
    np_im = np.array(im)
    
    return np_im

In [5]:
cat_file_list = glob.glob(dir_path_cat) #Busca en el directorio dado todas las imagenes que alli se encuentran
dog_file_list = glob.glob(dir_path_dog)
print(len(cat_file_list))
print(len(dog_file_list))

12501
12501


In [6]:
valid_size = [IMG_SIZE[1], IMG_SIZE[0], 3] 

def create_set(file_list, nItems):
    new_set = []
    for im in file_list[:nItems]:
       new_item = pixels_from_path(im)
       if len(new_item.shape) != 3:
            print('Imagen con tamano diferente', im)
       else:
          new_set.append(new_item)
    
    return new_set   

In [7]:
SAMPLE_SIZE = 2048

print("loading training cat images...")
cat_train_set = create_set( cat_file_list, SAMPLE_SIZE)

print("loading training dog images...")
dog_train_set = create_set(dog_file_list, SAMPLE_SIZE)


loading training cat images...
Imagen con tamano diferente ./PetImages\Cat\10125.jpg
Imagen con tamano diferente ./PetImages\Cat\10501.jpg
Imagen con tamano diferente ./PetImages\Cat\10820.jpg
Imagen con tamano diferente ./PetImages\Cat\11095.jpg
Imagen con tamano diferente ./PetImages\Cat\11210.jpg
Imagen con tamano diferente ./PetImages\Cat\11565.jpg
loading training dog images...
Imagen con tamano diferente ./PetImages\Dog\10158.jpg
Imagen con tamano diferente ./PetImages\Dog\10401.jpg
Imagen con tamano diferente ./PetImages\Dog\10747.jpg
Imagen con tamano diferente ./PetImages\Dog\10797.jpg
Imagen con tamano diferente ./PetImages\Dog\11410.jpg
Imagen con tamano diferente ./PetImages\Dog\11675.jpg


In [8]:
print(len(cat_train_set))
print(len(dog_train_set))

2042
2042


In [9]:
TEST_SIZE = 512
cat_file_list_test =  cat_file_list[SAMPLE_SIZE+1:]
cat_test_set = create_set(cat_file_list_test, TEST_SIZE)

dog_file_list_test =  dog_file_list[SAMPLE_SIZE+1:]
dog_test_set = create_set(dog_file_list_test, TEST_SIZE)

Imagen con tamano diferente ./PetImages\Cat\11874.jpg
Imagen con tamano diferente ./PetImages\Cat\11935.jpg
Imagen con tamano diferente ./PetImages\Cat\12080.jpg
Imagen con tamano diferente ./PetImages\Dog\11849.jpg
Imagen con tamano diferente ./PetImages\Dog\11853.jpg


In [10]:
print(len(cat_test_set))
print(len(dog_test_set))

509
510


In [11]:
def validate_size(im_list):
   valid_size = (IMG_SIZE[1], IMG_SIZE[0], 3) 
   for im in im_list:
       im_shape = im.shape 
       #if not (np.array(im_shape) == np.array(valid_size)).all():
       if not (np.array_equal(np.array(im_shape), np.array(valid_size))):
          print('hay una imagen con un tamano:', im_shape)
          im_list.remove(im)
   return im_list

In [12]:
cat_train_set = validate_size(cat_train_set)
dog_train_set = validate_size(dog_train_set)

cat_test_set = validate_size(cat_test_set)
dog_test_set = validate_size(dog_test_set)

labels_cat_train = np.ones(len(cat_train_set))
labels_dog_train = np.zeros(len(dog_train_set))

labels_cat_test = np.ones(len(cat_test_set))
labels_dog_test = np.zeros(len(dog_test_set))

train_set =  cat_train_set  + dog_train_set
y_train = np.concatenate((labels_cat_train, labels_dog_train), axis=None)
 
test_set =  cat_test_set  + dog_test_set
y_test = np.concatenate((labels_cat_test, labels_dog_test), axis=None)

print(len(train_set))
print(len(test_set))
print(len(y_train))
print(len(y_test))

hay una imagen con un tamano: (125, 94, 4)
4083
1019
4083
1019


<ipython-input-11-eca6d32de37c>:8: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  im_list.remove(im)


In [13]:
X_train = np.array(train_set)
X_test = np.array(test_set)

print(X_train.shape)
print(X_test.shape)

(4083, 125, 94, 3)
(1019, 125, 94, 3)


# Un MLP normal

In [14]:
from tensorflow import keras
from tensorflow.keras import layers

img_size = IMG_SIZE

total_pixels = img_size[0] *img_size[1] * 3
fc_size = 512

#Definicion del modelo

inputs = keras.Input(shape=(img_size[1], img_size[0],3), name='ani_image') 
x = layers.Flatten(name = 'flattened_img')(inputs) #turn image to vector.

x = layers.Dense(fc_size, activation='relu', name='first_layer')(x)
outputs = layers.Dense(1, activation='sigmoid', name='class')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [15]:
customAdam = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=customAdam,  # Optimizer
              # Loss function to minimize
              loss="mean_squared_error",
              # List of metrics to monitor
              metrics=["accuracy","mean_squared_error"])

In [16]:
print('# Fit model on training data')

history = model.fit(X_train, 
                    y_train,
                    batch_size=32, 
                    shuffle = True, #important since we loaded cats first, dogs second.
                    epochs=10,
                    validation_data=(X_test, y_test))

# Fit model on training data
Epoch 1/10
128/128 [==============================] - 7s 53ms/step - loss: 0.5000 - accuracy: 0.4999 - mean_squared_error: 0.5000 - val_loss: 0.5005 - val_accuracy: 0.4995 - val_mean_squared_error: 0.5005
Epoch 2/10
128/128 [==============================] - 7s 52ms/step - loss: 0.4999 - accuracy: 0.5001 - mean_squared_error: 0.4999 - val_loss: 0.5005 - val_accuracy: 0.4995 - val_mean_squared_error: 0.5005
Epoch 3/10
128/128 [==============================] - 7s 52ms/step - loss: 0.4999 - accuracy: 0.5001 - mean_squared_error: 0.4999 - val_loss: 0.5005 - val_accuracy: 0.4995 - val_mean_squared_error: 0.5005
Epoch 4/10
128/128 [==============================] - 7s 51ms/step - loss: 0.4999 - accuracy: 0.5001 - mean_squared_error: 0.4999 - val_loss: 0.5005 - val_accuracy: 0.4995 - val_mean_squared_error: 0.5005
Epoch 5/10
128/128 [==============================] - 7s 51ms/step - loss: 0.4999 - accuracy: 0.5001 - mean_squared_error: 0.4999 - val_loss: 0.5005 - 

# Single Convolutional Layer

In [17]:
fc_layer_size = 128
img_size = IMG_SIZE

conv_inputs = keras.Input(shape=(img_size[1], img_size[0],3), name='ani_image')
conv_layer = layers.Conv2D(24, kernel_size=3, activation='relu')(conv_inputs)
conv_layer = layers.MaxPool2D(pool_size=(2,2))(conv_layer)
conv_x = layers.Flatten(name = 'flattened_features')(conv_layer) #turn image to vector.

conv_x = layers.Dense(fc_layer_size, activation='relu', name='first_layer')(conv_x)
conv_x = layers.Dense(fc_layer_size, activation='relu', name='second_layer')(conv_x)
conv_outputs = layers.Dense(1, activation='sigmoid', name='class')(conv_x)

conv_model = keras.Model(inputs=conv_inputs, outputs=conv_outputs)

In [18]:
customAdam = keras.optimizers.Adam(lr=1e-6)
conv_model.compile(optimizer=customAdam,  # Optimizer
              # Loss function to minimize
              loss="binary_crossentropy",
              # List of metrics to monitor
              metrics=["accuracy","mean_squared_error"])

In [19]:
print('# Fit model on training data')

history = conv_model.fit(X_train, 
                    y_train, #we pass it th labels
                    #If the model is taking forever to train, make this bigger
                    #If it is taking forever to load for the first epoch, make this smaller
                    batch_size=64, 
                    shuffle = True,
                    epochs=10,
                    # We pass it validation data to
                    # monitor loss and metrics
                    # at the end of each epoch
                    validation_data=(X_test, y_test))

# Fit model on training data
Epoch 1/10
64/64 [==============================] - 8s 118ms/step - loss: 5.3044 - accuracy: 0.5295 - mean_squared_error: 0.4414 - val_loss: 3.4065 - val_accuracy: 0.5653 - val_mean_squared_error: 0.4004
Epoch 2/10
64/64 [==============================] - 8s 122ms/step - loss: 2.9436 - accuracy: 0.5496 - mean_squared_error: 0.4029 - val_loss: 2.5690 - val_accuracy: 0.5692 - val_mean_squared_error: 0.3876
Epoch 3/10
64/64 [==============================] - 8s 119ms/step - loss: 2.3096 - accuracy: 0.5643 - mean_squared_error: 0.3766 - val_loss: 2.3560 - val_accuracy: 0.5604 - val_mean_squared_error: 0.3883
Epoch 4/10
64/64 [==============================] - 8s 123ms/step - loss: 1.9373 - accuracy: 0.5827 - mean_squared_error: 0.3583 - val_loss: 2.1173 - val_accuracy: 0.5496 - val_mean_squared_error: 0.3838
Epoch 5/10
64/64 [==============================] - 8s 121ms/step - loss: 1.6439 - accuracy: 0.5822 - mean_squared_error: 0.3440 - val_loss: 1.9471 - val_a

# Bigger Convolutional Model

In [20]:
fc_layer_size = 256
img_size = IMG_SIZE

conv_inputs = keras.Input(shape=(img_size[1], img_size[0],3), name='ani_image')

conv_layer = layers.Conv2D(48, kernel_size=3, activation='relu')(conv_inputs)
conv_layer = layers.MaxPool2D(pool_size=(2,2))(conv_layer)

conv_layer = layers.Conv2D(48, kernel_size=3, activation='relu')(conv_layer)
conv_layer = layers.MaxPool2D(pool_size=(2,2))(conv_layer)

conv_x = layers.Flatten(name = 'flattened_features')(conv_layer) #turn image to vector.

conv_x = layers.Dense(fc_layer_size, activation='relu', name='first_layer')(conv_x)
conv_x = layers.Dense(fc_layer_size, activation='relu', name='second_layer')(conv_x)
conv_outputs = layers.Dense(1, activation='sigmoid', name='class')(conv_x)

conv_model = keras.Model(inputs=conv_inputs, outputs=conv_outputs)

In [21]:
customAdam = keras.optimizers.Adam(lr=1e-6)
conv_model.compile(optimizer=customAdam,  # Optimizer
              # Loss function to minimize
              loss="binary_crossentropy",
              # List of metrics to monitor
              metrics=["accuracy","mean_squared_error"])

In [22]:
print('# Fit model on training data')

history = conv_model.fit(X_train, 
                    y_train, 
                    batch_size=64, 
                    shuffle = True,
                    epochs=15,
                    # We pass it validation data to
                    # monitor loss and metrics
                    # at the end of each epoch
                    validation_data=(X_test, y_test))

# Fit model on training data
Epoch 1/15
64/64 [==============================] - 20s 312ms/step - loss: 2.7832 - accuracy: 0.5133 - mean_squared_error: 0.4269 - val_loss: 2.3080 - val_accuracy: 0.5201 - val_mean_squared_error: 0.4103
Epoch 2/15
64/64 [==============================] - 20s 318ms/step - loss: 1.9395 - accuracy: 0.5471 - mean_squared_error: 0.3821 - val_loss: 1.9177 - val_accuracy: 0.5329 - val_mean_squared_error: 0.3912
Epoch 3/15
64/64 [==============================] - 20s 316ms/step - loss: 1.6229 - accuracy: 0.5685 - mean_squared_error: 0.3555 - val_loss: 1.7054 - val_accuracy: 0.5319 - val_mean_squared_error: 0.3806
Epoch 4/15
64/64 [==============================] - 20s 313ms/step - loss: 1.4441 - accuracy: 0.5805 - mean_squared_error: 0.3390 - val_loss: 1.6130 - val_accuracy: 0.5339 - val_mean_squared_error: 0.3718
Epoch 5/15
64/64 [==============================] - 20s 318ms/step - loss: 1.3245 - accuracy: 0.5942 - mean_squared_error: 0.3237 - val_loss: 1.5306 - 

# Bigger Model2

In [23]:
fc_layer_size = 256
img_size = IMG_SIZE

conv_inputs = keras.Input(shape=(img_size[1], img_size[0],3), name='ani_image')
conv_layer = layers.Conv2D(128, kernel_size=3, activation='relu')(conv_inputs)
conv_layer = layers.MaxPool2D(pool_size=(2,2))(conv_layer)

conv_layer = layers.Conv2D(128, kernel_size=3, activation='relu')(conv_layer)
conv_layer = layers.MaxPool2D(pool_size=(2,2))(conv_layer)

conv_x = layers.Flatten(name = 'flattened_features')(conv_layer) #turn image to vector.

conv_x = layers.Dense(fc_layer_size, activation='relu', name='first_layer')(conv_x)
conv_x = layers.Dense(fc_layer_size, activation='relu', name='second_layer')(conv_x)
conv_outputs = layers.Dense(1, activation='sigmoid', name='class')(conv_x)

huge_conv_model = keras.Model(inputs=conv_inputs, outputs=conv_outputs)

In [24]:
customAdam = keras.optimizers.Adam(lr=1e-6)
huge_conv_model.compile(optimizer=customAdam,  # Optimizer
              # Loss function to minimize
              loss="binary_crossentropy",
              # List of metrics to monitor
              metrics=["accuracy","mean_squared_error"])

In [25]:
print('# Fit model on training data')

history = huge_conv_model.fit(X_train, 
                    y_train, #we pass it th labels
                    #If the model is taking forever to train, make this bigger
                    #If it is taking forever to load for the first epoch, make this smaller
                    batch_size=64, 
                    shuffle = True,
                    epochs=10,
                    # We pass it validation data to
                    # monitor loss and metrics
                    # at the end of each epoch
                    validation_data=(X_test, y_test))

# Fit model on training data
Epoch 1/10
64/64 [==============================] - 66s 1s/step - loss: 2.0869 - accuracy: 0.5016 - mean_squared_error: 0.4038 - val_loss: 1.2765 - val_accuracy: 0.5289 - val_mean_squared_error: 0.3652
Epoch 2/10
64/64 [==============================] - 67s 1s/step - loss: 1.0677 - accuracy: 0.5347 - mean_squared_error: 0.3325 - val_loss: 1.0719 - val_accuracy: 0.5397 - val_mean_squared_error: 0.3318
Epoch 3/10
64/64 [==============================] - 68s 1s/step - loss: 0.9414 - accuracy: 0.5667 - mean_squared_error: 0.3018 - val_loss: 1.0066 - val_accuracy: 0.5643 - val_mean_squared_error: 0.3165
Epoch 4/10
64/64 [==============================] - 68s 1s/step - loss: 0.8356 - accuracy: 0.6023 - mean_squared_error: 0.2732 - val_loss: 1.0112 - val_accuracy: 0.5653 - val_mean_squared_error: 0.3157
Epoch 5/10
64/64 [==============================] - 68s 1s/step - loss: 0.7666 - accuracy: 0.6221 - mean_squared_error: 0.2548 - val_loss: 0.9128 - val_accuracy: 0

# Modelo de Jason Brownlee

In [31]:
fc_layer_size = 256
img_size = IMG_SIZE

conv_inputs = keras.Input(shape=(img_size[1], img_size[0],3), name='ani_image')

conv_layer = layers.Conv2D(64, kernel_size=3, activation='relu',
                           kernel_initializer='he_uniform')(conv_inputs)
conv_layer = layers.MaxPool2D(pool_size=(2,2))(conv_layer)

conv_layer = layers.Dropout(0.2)(conv_layer)
conv_layer = layers.Conv2D(64, kernel_size=3, activation='relu',
                           kernel_initializer='he_uniform')(conv_layer)
conv_layer = layers.MaxPool2D(pool_size=(2,2))(conv_layer)

conv_layer = layers.Dropout(0.2)(conv_layer)
conv_layer = layers.Conv2D(64, kernel_size=3, activation='relu',
                           kernel_initializer='he_uniform')(conv_layer)
conv_layer = layers.MaxPool2D(pool_size=(2,2))(conv_layer)

conv_layer = layers.Dropout(0.2)(conv_layer)
conv_x = layers.Flatten(name = 'flattened_features')(conv_layer) #turn image to vector.

conv_x = layers.Dense(fc_layer_size, activation='relu', name='first_layer',
                      kernel_initializer='he_uniform')(conv_x)
conv_x = layers.Dropout(0.5)(conv_x)
conv_x = layers.Dense(fc_layer_size, activation='relu', name='second_layer')(conv_x)
conv_outputs = layers.Dense(1, activation='sigmoid', name='class')(conv_x)

Brownlee_model = keras.Model(inputs=conv_inputs, outputs=conv_outputs)


In [32]:
customAdam = keras.optimizers.Adam(lr=1e-6)
Brownlee_model.compile(optimizer=customAdam,  # Optimizer
              # Loss function to minimize
              loss="binary_crossentropy",
              # List of metrics to monitor
              metrics=["binary_crossentropy","mean_squared_error"])

In [33]:
print('# Fit model on training data')

history = Brownlee_model.fit(X_train, 
                    y_train, #we pass it th labels
                    #If the model is taking forever to train, make this bigger
                    #If it is taking forever to load for the first epoch, make this smaller
                    batch_size=64, 
                    shuffle = True,
                    epochs=10,                             
                    # We pass it validation data to
                    # monitor loss and metrics
                    # at the end of each epoch
                    validation_data=(X_test, y_test))

# Fit model on training data
Epoch 1/5
64/64 [==============================] - 40s 617ms/step - loss: 148.5111 - binary_crossentropy: 148.5111 - mean_squared_error: 0.4880 - val_loss: 32.5123 - val_binary_crossentropy: 32.5123 - val_mean_squared_error: 0.4938
Epoch 2/5
64/64 [==============================] - 41s 643ms/step - loss: 128.4026 - binary_crossentropy: 128.4026 - mean_squared_error: 0.5074 - val_loss: 29.5024 - val_binary_crossentropy: 29.5024 - val_mean_squared_error: 0.4945
Epoch 3/5
64/64 [==============================] - 40s 632ms/step - loss: 120.4583 - binary_crossentropy: 120.4583 - mean_squared_error: 0.4959 - val_loss: 28.5973 - val_binary_crossentropy: 28.5973 - val_mean_squared_error: 0.4930
Epoch 4/5
64/64 [==============================] - 40s 629ms/step - loss: 112.6544 - binary_crossentropy: 112.6544 - mean_squared_error: 0.4951 - val_loss: 26.3280 - val_binary_crossentropy: 26.3280 - val_mean_squared_error: 0.4864
Epoch 5/5
64/64 [==========================

# Transfer learning

## VGG-16

In [ ]:
# define cnn model
import tensorflow as tf
from tensorflow import keras 

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

img_size = IMG_SIZE

model_VGG16 = VGG16(include_top=False, input_shape=(img_size[1], img_size[0],3))
for layer in model_VGG16.layers:
   layer.trainable = False

flat1  = Flatten()(model_VGG16.layers[-1].output)
class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
output = Dense(1, activation='sigmoid')(class1)

model_VGG16 = Model(inputs=model_VGG16.inputs, outputs=output)

customAdam = keras.optimizers.Adam(lr=1e-6)
model_VGG16.compile(optimizer=customAdam,  # Optimizer
              # Loss function to minimize
              loss="binary_crossentropy",
              # List of metrics to monitor
              metrics=["binary_crossentropy","mean_squared_error"])


history = model_VGG16.fit(X_train, 
                    y_train, #we pass it th labels
                    #If the model is taking forever to train, make this bigger
                    #If it is taking forever to load for the first epoch, make this smaller
                    batch_size = 64, 
                    shuffle = True,
                    epochs=10,                             
                    # We pass it validation data to
                    # monitor loss and metrics
                    # at the end of each epoch
                    validation_data=(X_test, y_test))


Epoch 1/10
64/64 [==============================] - 121s 2s/step - loss: 5.4601 - binary_crossentropy: 5.4601 - mean_squared_error: 0.3847 - val_loss: 5.3756 - val_binary_crossentropy: 5.3756 - val_mean_squared_error: 0.3758
Epoch 2/10
64/64 [==============================] - 121s 2s/step - loss: 4.8114 - binary_crossentropy: 4.8114 - mean_squared_error: 0.3669 - val_loss: 4.7586 - val_binary_crossentropy: 4.7586 - val_mean_squared_error: 0.3695
Epoch 3/10
64/64 [==============================] - 122s 2s/step - loss: 4.3781 - binary_crossentropy: 4.3781 - mean_squared_error: 0.3536 - val_loss: 4.3236 - val_binary_crossentropy: 4.3236 - val_mean_squared_error: 0.3608
Epoch 4/10
64/64 [==============================] - 120s 2s/step - loss: 4.0675 - binary_crossentropy: 4.0675 - mean_squared_error: 0.3430 - val_loss: 4.0012 - val_binary_crossentropy: 4.0012 - val_mean_squared_error: 0.3480
Epoch 5/10
64/64 [==============================] - 122s 2s/step - loss: 3.8242 - binary_crossentrop